In [1]:
import os

In [16]:
with open('tiny.txt', 'r', encoding = 'utf-8') as f:
    text = f.read()

In [17]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [7]:
chars = sorted(list(set(data)))
print(''.join(chars))


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [8]:
print(len(chars))

65


In [13]:
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}

encode = lambda s: [stoi[ch] for ch in s]
decode = lambda x: ''.join([itos[i] for i in x])

print(encode("hello"))
print(decode([46, 43, 50, 50, 53]))

[46, 43, 50, 50, 53]
hello


In [15]:
import torch

In [19]:
data = torch.tensor(encode(text), dtype = torch.long)
print(data.shape, data.dtype)

print(data[:100])

torch.Size([1115393]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [22]:
train_size = int(data.shape[0] * 0.9)
train_data = data[:train_size]
val_data = data[train_size:]

In [26]:
batch_size = 32
block_size = 8
vocab_size = len(chars)

In [31]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data)- block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')

print(xb.shape, yb.shape)

torch.Size([32, 8]) torch.Size([32, 8])


In [37]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_layer = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets = None):
        logits = self.token_embedding_layer(idx)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:,-1,:]
            probs = F.softmax(logits, dim = -1)
            idx_next = torch.multinomial(probs, num_samples = 1)
            idx = torch.cat([idx, idx_next], dim = 1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape, loss)

torch.Size([256, 65]) tensor(4.8127, grad_fn=<NllLossBackward0>)


In [39]:
idx = torch.zeros((1, 1), dtype=torch.long)
print(decode(m.generate(idx, 100)[0].tolist()))


Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [40]:
optimizer = torch.optim.AdamW(m.parameters(), lr = 1e-3)

In [47]:
for steps in range(10000):
    xb, yb = get_batch('train')
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

2.478039026260376


In [48]:
print(decode(m.generate(idx, 400)[0].tolist()))


Hacer XI:
Fo nanesiet ICl.
Durit ben, ute pake p, saneiso fathin' woie mmespry mmarspilishee o my me ck, by my! olelewoure ite t

MAlartso d l telllenng at:
ING nor, he RETIUSt mar worn swangren, o--ombed asen heverelaty se onde
Co,
sand co ak d, fort
TEYOve
O:
Me, w; wnatagr, hive
THen yice on ond, d f, t bushe beg corallas weas--
Y fr engheeangde direl t,
Ant hatan adgixcanitour knerethe isit:
O
